In [17]:
from itertools import compress
import pandas as pd
import re
import os

In [18]:
def re_match(include, no_space_name):
    include_pattern = re.compile(include)
    return bool(include_pattern.search(no_space_name))

In [19]:
def match_mapping(rule, skuname):
    rule_include = rule['rule_include']
    rule_equal = rule['rule_equal']
    rule_exclude = rule['rule_exclude']
    output = rule['output']
    match_list = []
    for name in skuname:
        if type(name)!=str:
            continue
        no_space_name = name.replace(" ","")
        result_include = all([re_match(include.lower(), no_space_name.lower()) for include in rule_include])
        result_equal = all([equal.lower() == no_space_name.lower() for equal in rule_equal])
        result_exclude = True
        if len(rule_exclude) >= 1 and rule_exclude[0] != '':
            result_exclude = all([not re_match(exclude.lower(), no_space_name.lower()) for exclude in rule_exclude])
        result = (result_include or result_equal) and result_exclude
        if result:
            match_list.append({"title":name, "skuname": output})
    return match_list

In [20]:
def extract_group(group):
    result_group = ''
    rule_in_group = re.findall(r'\((.*?)\)',group)
    if len(rule_in_group)>0:
        result_group = rule_in_group[0]
    else:
        result_group = group
    return result_group

In [21]:
def get_rule(row):
    rule_include = [extract_group(n) for n in row['include'].split('&')]
    rule_equal = [extract_group(n) for n in row['exclude'].split('|')]
    rule_exclude = [extract_group(n) for n in row['exclude'].split('&')]
    return {'rule_include': rule_include, 'rule_equal':rule_equal, 'rule_exclude':rule_exclude, 'output':row['output']}

In [22]:
def write_to_csv(mapping_df,path):
    if not os.path.isfile(path):
        mapping_df.to_csv(path, encoding='utf8', mode='w', header=True, index=False)
    else:
        mapping_df.to_csv(path, encoding='utf8', mode='a', header=False, index=False)

In [23]:
def main():
    skumapping_path = '/Users/dataone/MARYKAY/SKUmapping'
    sku_path = skumapping_path + '/name_origin_test.csv'
    rule_path =  skumapping_path + '/auto_mapping_rule.csv'
    result_path =  skumapping_path + '/name_origin_test_result.csv'
    dup_result_path =  skumapping_path + '/name_origin_test_result_dup.csv'
    no_match_path = skumapping_path + '/name_origin_test_no_match_mapping_result.csv'
    origin_brands = pd.read_csv(brands_path)
    skuname = ['幻时抗皱精华素']
    # rule_file = pd.read_excel('/Users/duanyujia/dataone/mars/相等测试规则.xlsx')
    # rule_group=rule_file.replace(to_replace='\ufeff',value='',regex=True)
    rule_group = pd.read_csv(rule_path,na_filter=False)
    result = []
    for index, row in rule_group.iterrows():
        rule = get_rule(row)
        match_result = match_mapping(rule, skuname)
        if len(match_result) > 0:
            result += match_result
    df_result =  pd.DataFrame(result).drop_duplicates()
    grouped = df_result.groupby(['origin_title'])
    ready_result = grouped.filter(lambda x: x['origin_title'].agg(['count']) == 1)
    write_to_csv(ready_result, result_path)
    confuse_result = grouped.filter(lambda x: x['origin_title'].agg(['count']) > 1)
    write_to_csv(confuse_result, dup_result_path)
    mapped_origin_brands = df_result['origin_title']
    no_mapped_brands = pd.DataFrame({'title_origin': list(set(skuname) - set(mapped_origin_sku))})
    write_to_csv(no_mapped_skus, no_match_path)

In [24]:
if __name__ == '__main__':
    main()